In [1]:
import numpy as np
import meshio
import pyvista as pv
from pathlib import Path
import os
import sys
import vedo 
import tempfile


In [2]:

def vm2xdmf(fname_pts,fname_elm,fname_vm=None):

    if os.path.splitext(fname_pts)[1] == ".bpts":
        with open(fname_pts,"rb") as fi:
            # first 1024 is header in string format
            npts = int( fi.read(1024).split()[0] )
            pts = np.fromfile(fi,dtype=np.float32).reshape((npts,-1))
    elif os.path.splitext(fname_pts)[1] == ".pts":
        # first row is number of points
        pts = np.loadtxt(fname_pts,dtype=np.float32,skiprows=1)
    else:
        raise RuntimeError(f"File format of {fname_pts} unknown!")
    
    npts = pts.shape[0]  # number of points
    gdim = pts.shape[1]  # geometrical dimension

    if os.path.splitext(fname_elm)[1] == ".belem":
        with open(fname_elm,"rb") as fi:
            # first 1024 is header in string format
            nelm = int( fi.read(1024).split()[0] )
            elm  = np.fromfile(fi,dtype=np.int32).reshape((nelm,-1))
            elm  = elm[:,1:-1]   # remove first and last column
    elif os.path.splitext(fname_elm)[1] == ".elem":
        with open(fname_elm,"r") as fi:
            nelm  = int(fi.readline().strip())
            ncols = len(fi.readline().split())
        elm = np.loadtxt(fname_elm,dtype=np.int32,skiprows=1,usecols=range(1,ncols-1))
        cell_id = np.loadtxt(fname_elm,dtype=np.int32,skiprows=1,usecols=[ncols-1])
    else:
        raise RuntimeError(f"File format of {fname_elm} unknown!")
    
    nelm = elm.shape[0]  # number of elements
    tdim = elm.shape[1]  # topological dimension

    cells = {}
    cells[{4:'tetra',3:'triangle'}[tdim]] = elm

    
    elm = np.array(elm)
    elm = np.c_[np.ones(len(elm))*10, elm]
    elm = elm.astype(int)
    pts = np.array(pts)
    
    with tempfile.TemporaryDirectory() as fdir_out:
        fname_out = Path(fdir_out) / 'temp.vtu'
        print(f"Writing {fname_out}...")
        if fname_vm is not None:
            vm = read_igb(fname_vm).squeeze()
            print(vm.shape)

            with meshio.xdmf.TimeSeriesWriter(fname_out) as writer:
                writer.write_points_cells(pts, cells)
                for t in range(vm.shape[0]):
                    writer.write_data(float(t), point_data={"vm": vm[t,:]})

        else:
            meshio.Mesh(points=pts, cells=cells).write(fname_out)

        mesh = pv.read(fname_out)
    mesh['CellId'] = cell_id
    
    if fname_elm.with_suffix(".extra").is_file():
        extra = np.loadtxt(fname_elm.with_suffix(".extra"), dtype=int)[1:]
        mesh['extra'] = np.isin(cell_id, extra)

    if fname_elm.with_suffix(".intra").is_file():
        intra = np.loadtxt(fname_elm.with_suffix(".intra"), dtype=int)[1:]
        mesh['intra'] = np.isin(cell_id, intra)

    if fname_elm.with_suffix(".part").is_file():
        part =  np.loadtxt(fname_elm.with_suffix(".part"), dtype=int)[1:]
        mapping = dict(zip(part[::2], part[1::2]))
        mesh['part'] = np.vectorize(mapping.get)(cell_id)

    # mesh.save(fname_out)
    return mesh
    

In [3]:
fdir = Path("/home/arsenii/Documents/PYTHON/MICROCARD/microcard-bench/data/emigrid/carp")
# # fdir = Path("/home/arsenii/Documents/CPP/MICROCARD/WP7/synthetic/output")
# fdir = Path('/mnt/89ea702f-c213-4e9a-acb8-8c72a41b672e/Documents/CPP/MICROCARD2/Grech/extend-domain-large')
# # fdir = Path("/mnt/89ea702f-c213-4e9a-acb8-8c72a41b672e/Documents/CPP/MICROCARD2/Grech/output/emiGrid_x4_y4_z4_r1")

# fdir = Path('/mnt/89ea702f-c213-4e9a-acb8-8c72a41b672e/Documents/CPP/MICROCARD2/d7.2-strong-scaling/robin-4-62746')


In [4]:
# fname = Path(f"emiGrid_x4_y4_z4_r1.elem")
# fname = Path(f"rmtest-6x5x4.elem")
# fname = Path('robin-4-62746.elem')
fname = Path("emigrid.elem")

fname_elm = fdir/fname
fname_pts = fdir/fname.with_suffix(".pts")
fname_vm  = None
fname_out = fdir/fname.with_suffix('.vtu')

mesh = vm2xdmf(fname_pts,fname_elm,fname_vm)

# mesh = pv.read(fname_out)
# mesh['CellId'] = cell_id

Writing /tmp/tmpvds9tb8a/temp.vtu...


In [5]:
print(np.unique(mesh['CellId'][mesh['intra']]).shape)
print(np.unique(mesh['CellId'][mesh['extra']]).shape)

(27,)
(27,)


In [6]:
mesh.plot(scalars='intra')
# mesh.save(fdir/fname.with_suffix(".vtu"))

Widget(value='<iframe src="http://localhost:34165/index.html?ui=P_0x7e188ad93e00_0&reconnect=auto" class="pyvi…

In [20]:
mesh.n_points

179036

In [9]:
def read_igb_header(filename):
    "Read and parse the header of the IGB file"

    if filename is None:
        raise RuntimeError("No filename specified")

    # the header is the first 1024-chunk of the file
    # it's just a string, easy to read and parse
    #
    with open(filename,"rb") as pdata:
        buf = pdata.read(1024)

        hdr = buf.decode().split('\r\n')
        cmm = [l.strip()[2:] for l in hdr if l.startswith("#")]
        hdr = sum((l.split() for l in (l.strip() for l in hdr if not l.startswith("#")) if len(l)>0),[])
        hdr = dict(tuple(ob.split(":")) for ob in hdr)
        # FIXME endianness
        hdr['comments'] = cmm
        for a in 'xyzt':
            if a in hdr: hdr[a] = int(hdr[a])
        for a in ['zero','facteur']:
            if a in hdr: hdr[a] = float(hdr[a])

    return hdr

def read_igb(filename,convert_to_float=False,return_header=False):
    "Read IGB file into numpy format"

    dtypes = { 'byte':  np.uint8,   'char': np.int8,
               'short': np.int16,   'long': np.int32,
               'float': np.float32, 'double': np.float64 }

    hdr = read_igb_header(filename)
    nx,ny,nz = hdr['x'],hdr['y'],hdr['z']
    nt = hdr.get('t',1)
    shape = (nt,nz,ny,nx) if nt > 1 else (nz,ny,nx)
    dtype = dtypes[hdr['type']]
    print(f"{shape=}")
    
    data = np.fromfile(filename,dtype=dtype,count=nx*ny*nz*nt,offset=1024).reshape(shape)

    if convert_to_float:
        data = hdr.get('facteur',1.0)*data + hdr.get('zero',0.0)

    if return_header:
        return data, hdr
    else:   
        return data

fdir = Path("/home/arsenii/Documents/PYTHON/MICROCARD/microcard-bench/output/emigrid")
data = read_igb(fdir/"vm.igb")
data

shape=(151, 1, 1, 131572)


array([[[[-80.      , -80.      , -80.      , ..., -80.      ,
          -80.      , -80.      ]]],


       [[[ 19.998302,  19.998302,  19.998322, ...,  19.99752 ,
           19.997513,  19.997513]]],


       [[[ 19.995522,  19.995522,  19.99554 , ...,  19.99473 ,
           19.99472 ,  19.99472 ]]],


       ...,


       [[[ 18.577328,  18.577328,  18.577484, ...,  18.570648,
           18.57058 ,  18.57058 ]]],


       [[[ 18.553375,  18.553375,  18.553534, ...,  18.546595,
           18.546524,  18.546524]]],


       [[[ 18.529057,  18.529057,  18.529217, ...,  18.522171,
           18.5221  ,  18.5221  ]]]],
      shape=(151, 1, 1, 131572), dtype=float32)

In [10]:
fname = Path(f"emigrid_m.elem")
# fname = Path(f"rmtest-6x5x4.elem")

fname_elm = fdir/fname
fname_pts = fdir/fname.with_suffix(".pts")
fname_vm  = None
fname_out = fdir/fname.with_suffix('.vtu')

mesh = vm2xdmf(fname_pts,fname_elm,fname_vm)

# mesh = pv.read(fname_out)
# mesh['CellId'] = cell_id

Writing /tmp/tmpbjgz36ir/temp.vtu...


In [9]:
# mesh.plot()

In [12]:
for i in range(len(data)):
# mesh['Vm'] = -80*np.ones(mesh.n_cells)
# for i in [0]:
    mesh['Vm'] = data[i, ...].squeeze()
    # mesh.plot(scalars='Vm')
    mesh.save(f"/tmp/mesh_vms/mesh_{str(i).zfill(3)}.vtu")

# (mesh['extra'])

In [11]:
data.max(axis=-1).squeeze()

array([2.00000000e+01, 5.53650326e-09, 1.21843598e-07, 8.03477960e-08,
       6.96076086e-08, 8.68363514e-08, 6.84660861e-08, 5.03914563e-08,
       5.00024910e-08, 7.19957285e-08, 4.84304188e-08, 4.72852122e-08,
       4.22464872e-08, 3.91531714e-08, 4.31787690e-08, 3.64470232e-08,
       4.48241728e-08, 6.42524398e-08, 6.68609061e-08, 2.84200645e-08,
       3.66269042e-08, 2.46151135e-08, 3.16444506e-08, 4.21736139e-08,
       1.52881192e-08, 4.19682777e-08, 2.52635566e-08, 2.59365560e-08,
       8.50049275e-09, 4.39902159e-08, 2.95868769e-08, 1.23675568e-08,
       2.87870829e-08, 1.01842561e-08, 4.73851607e-08, 2.66942646e-08,
       3.74173972e-08, 2.72678928e-08, 3.81764416e-08, 5.28152810e-08,
       9.50450119e-09, 5.15758458e-08, 3.83005840e-08, 2.30839134e-08,
       6.11304998e-08, 1.67269487e-08, 6.85269796e-08, 1.71570811e-08,
       1.75383104e-08, 7.34514671e-09, 1.92226626e-08, 5.31178372e-08,
       1.85924165e-08, 1.25110091e-07, 8.26356725e-08, 4.22254480e-08,
      

In [12]:
# mesh['Vm'][mesh['intra']]
# mesh['intra']

np.sum(mesh['intra']), np.sum(mesh['extra']), mesh.n_cells

KeyError: 'Data array (intra) not present in this dataset.'